# Building Fast Queries on a CSV

Excercise to practice writing fast queries on CSV.

## Brief description

Data is a csv file (from Kaggle) of laptops descriptions and prices. 

Description of the rows:

* **ID:** A unique identifier for the laptop.
* **Company:** The name of the company that produces the laptop.
* **Product:** The name of the laptop.
* **TypeName:** The type of laptop.
* **Inches:** The size of the screen in inches.
* **ScreenResolution:** The resolution of the screen.
* **CPU:** The laptop CPU.
* **RAM:** The amount of RAM in the laptop.
* **Memory:** The size of the hard drive.
* **GPU:** The graphics card name.
* **OpSys:** The name of the operating system.
* **Weight:** The laptop weight.
* **Price:** The price of the laptop.

In [1]:
# Read the data
import csv

with open('laptops.csv') as file:
    reader = csv.reader(file)
    rows = list(reader)
    header = rows[0]
    rows = rows[1:]
    
print(header)
print('')
print(rows[:5])

['Id', 'Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price']

[['6571244', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 'IPS Panel Retina Display 2560x1600', 'Intel Core i5 2.3GHz', '8GB', '128GB SSD', 'Intel Iris Plus Graphics 640', 'macOS', '1.37kg', '1339'], ['7287764', 'Apple', 'Macbook Air', 'Ultrabook', '13.3', '1440x900', 'Intel Core i5 1.8GHz', '8GB', '128GB Flash Storage', 'Intel HD Graphics 6000', 'macOS', '1.34kg', '898'], ['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', '575'], ['9722156', 'Apple', 'MacBook Pro', 'Ultrabook', '15.4', 'IPS Panel Retina Display 2880x1800', 'Intel Core i7 2.7GHz', '16GB', '512GB SSD', 'AMD Radeon Pro 455', 'macOS', '1.83kg', '2537'], ['8550527', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 'IPS Panel Retina Display 2560x1600', 'Intel Core i5 3.1GHz', '8GB', '256GB

## Inventory Class
Goal is to create a class that represents the inventory. 

Requirements
* Given a laptop id, find the corresponding data.
* Given an amount of money, find whether there are two laptops whose total price is that given amount.
* Identify all laptops whose price falls within a given budget.

In [2]:
# Create the initial class
class Inventory():
    
    def __init__(self, csv_filename):
        with open(csv_filename) as file:
            reader = csv.reader(file)
            rows = list(reader)
        self.header = rows[0]
        self.rows = rows[1:]
        for row in self.rows:
            row[-1] = int(row[-1])

# Test the class
inventory = Inventory('laptops.csv')
print(inventory.header)
print(len(inventory.rows))

['Id', 'Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price']
1303


## Laptop ID Lookup

In [3]:
# Create a function to retrieve the laptop from the id
class Inventory():
    
    def __init__(self, csv_filename):
        with open(csv_filename) as file:
            reader = csv.reader(file)
            rows = list(reader)
        self.header = rows[0]
        self.rows = rows[1:]
        for row in self.rows:
            row[-1] = int(row[-1])
            
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None

# Test the function
inventory = Inventory('laptops.csv')
print(inventory.get_laptop_from_id('3362737'))
print(inventory.get_laptop_from_id('3362736'))
print(len(inventory.rows))

['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', 575]
None
1303


### Improve lookup with Preprocessing
The time complexity of the algorithm in the id lookup function is O(R) where R is the number of rows. We can decrease the time comoplexity by preprocessing the data (using a set or dictionary). In this case, we'll use a dictionary, which allows us to associate the id with its row (value and key).

In [4]:
# Preprocess the data by adding a dictionary to the __init__ method
# Then create a function to use that dictionary.
class Inventory():
    
    def __init__(self, csv_filename):
        with open(csv_filename) as file:
            reader = csv.reader(file)
            rows = list(reader)
        self.header = rows[0]
        self.rows = rows[1:]
        for row in self.rows:
            row[-1] = int(row[-1])
        # Add Dictionary here
        self.id_to_row = {}
        for row in self.rows:
            self.id_to_row[row[0]] = row
            
    def get_laptop_from_id_fast(self, laptop_id):
        # Function only requires lookup in the dicitonary
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None
    
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None
    
# Test the function
inventory = Inventory('laptops.csv')
print(inventory.get_laptop_from_id_fast('3362737'))
print(inventory.get_laptop_from_id_fast('3362736'))

['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', 575]
None


### Results of Preprocessing
The new lookup method has time complexity of O(1) (as compared to O(R) for the previous method), but it requires some additional memory and time creating the instance (because it creates a dictionary). The following will test the improvement using random numbers.

In [5]:
import time
import random

# Creating random ids for testing
ids = [str(random.randint(1000000, 9999999)) for _ in range(10000)]

# Create instance of the inventory
inventory = Inventory('laptops.csv')

# Find the time without preprocessing
total_time_no_dict = 0

for val in ids:
    start = time.time()
    inventory.get_laptop_from_id(val)
    end = time.time()
    total_time_no_dict += end - start
    
# Find the tiem with preprocessing
total_time_dict = 0

for val in ids:
    start = time.time()
    inventory.get_laptop_from_id_fast(val)
    end = time.time()
    total_time_dict += end - start
    
print('Without Preprocessing: ', total_time_no_dict)
print('Preprocessing: ', total_time_dict)
print('Time difference: ', total_time_no_dict - total_time_dict)
print('Improvement: ', round(total_time_no_dict / total_time_dict, 2), 'times better')

Without Preprocessing:  0.5610618591308594
Preprocessing:  0.0024056434631347656
Time difference:  0.5586562156677246
Improvement:  233.23 times better


## Price Lookup
In this hypothetical case, the store needs to lookup the price of laptops to ensure that any gift card issued can purchase one or two laptops in the inventory without leaving any remainder. The first function will make a simple price lookup with these constraints.

In [6]:
class Inventory():
    
    def __init__(self, csv_filename):
        with open(csv_filename) as file:
            reader = csv.reader(file)
            rows = list(reader)
        self.header = rows[0]
        self.rows = rows[1:]
        for row in self.rows:
            row[-1] = int(row[-1])
        self.id_to_row = {}
        for row in self.rows:
            self.id_to_row[row[0]] = row
            
    def get_laptop_from_id_fast(self, laptop_id):
        # Function only requires lookup in the dicitonary
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None
    
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None
    
    # Function for price lookup follows
    def check_promotion_dollars(self, dollars):
        # Matches the price of one laptop
        for row in self.rows:
            if row[-1] == dollars:
                return True
        # Matches the price of two laptops
        for row1 in self.rows:
            for row2 in self.rows:
                if row1[-1] + row2[-1] == dollars:
                    return True
        return False
    
# Test the function
inventory = Inventory('laptops.csv')
print(inventory.check_promotion_dollars(1000))
print(inventory.check_promotion_dollars(442))

True
False


### Improve Price Lookup with Preprocessing
By creating a set for prices when initializing the class, we can preprocess the data and improve the speed of the function.

In [7]:
# Add a set for preprocessing price lookup
class Inventory():
    
    def __init__(self, csv_filename):
        with open(csv_filename) as file:
            reader = csv.reader(file)
            rows = list(reader)
        self.header = rows[0]
        self.rows = rows[1:]
        for row in self.rows:
            row[-1] = int(row[-1])
        # Dictiorary for ids
        self.id_to_row = {}
        for row in self.rows:
            self.id_to_row[row[0]] = row
        # Set for prices
        self.prices = set()
        for row in self.rows:
            self.prices.add(row[-1])
            
    def get_laptop_from_id_fast(self, laptop_id):
        # Function only requires lookup in the dicitonary
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None
    
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None
    
    def check_promotion_dollars(self, dollars):
        # Matches the price of one laptop
        for row in self.rows:
            if row[-1] == dollars:
                return True
        # Matches the price of two laptops
        for row1 in self.rows:
            for row2 in self.rows:
                if row1[-1] + row2[-1] == dollars:
                    return True
        return False
    
    def check_promotion_dollars_fast(self, dollars):
        # Matches price of one laptop
        if dollars in self.prices:
            return True
        # Matches price of two laptops
        for val in self.prices:
            if dollars - val in self.prices:
                return True
        return False

# Test the function
inventory = Inventory('laptops.csv')
print(inventory.check_promotion_dollars_fast(1000))
print(inventory.check_promotion_dollars_fast(442))

True
False


### Results of preprocessing
The following will measure the improvement results of adding preprocessing.

In [8]:
# Creating random prices for testing
prices = [random.randint(100, 5000) for _ in range(100)]

# Create instance of the inventory
inventory = Inventory('laptops.csv')

# Find the time without preprocessing
total_time_no_set = 0

for price in prices:
    start = time.time()
    inventory.check_promotion_dollars(price)
    end = time.time()
    total_time_no_set += end - start
    
# Find the tiem with preprocessing
total_time_set = 0

for price in prices:
    start = time.time()
    inventory.check_promotion_dollars_fast(price)
    end = time.time()
    total_time_set += end - start
    
print('Without Preprocessing: ', total_time_no_set)
print('Preprocessing: ', total_time_set)
print('Time difference: ', total_time_no_set - total_time_set)
print('Improvement: ', round(total_time_no_set / total_time_set, 2), 'times better')

Without Preprocessing:  0.6684751510620117
Preprocessing:  0.00038623809814453125
Time difference:  0.6680889129638672
Improvement:  1730.73 times better


## Find Laptops within Budget
Create a function for finding all laptops within a given budget. First, sort the inventory by price. Then, use binary search to find where a given budget falls in the inventory.

In [9]:
# Function for finding the price in a row
def row_price(row):
    return row[-1]

# Sorting the inventory by price
class Inventory():
    
    def __init__(self, csv_filename):
        with open(csv_filename) as file:
            reader = csv.reader(file)
            rows = list(reader)
        self.header = rows[0]
        self.rows = rows[1:]
        for row in self.rows:
            row[-1] = int(row[-1])
        # Dictiorary for ids
        self.id_to_row = {}
        for row in self.rows:
            self.id_to_row[row[0]] = row
        # Set for prices
        self.prices = set()
        for row in self.rows:
            self.prices.add(row[-1])
        # Sort rows by price using row_price as key
        self.rows_by_price = sorted(self.rows, key=row_price)
            
    def get_laptop_from_id_fast(self, laptop_id):
        # Function only requires lookup in the dicitonary
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None
    
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None
    
    def check_promotion_dollars(self, dollars):
        # Matches the price of one laptop
        for row in self.rows:
            if row[-1] == dollars:
                return True
        # Matches the price of two laptops
        for row1 in self.rows:
            for row2 in self.rows:
                if row1[-1] + row2[-1] == dollars:
                    return True
        return False
    
    def check_promotion_dollars_fast(self, dollars):
        # Matches price of one laptop
        if dollars in self.prices:
            return True
        # Matches price of two laptops
        for val in self.prices:
            if dollars - val in self.prices:
                return True
        return False
    
    # Function for finding first laptop in inventory more expensive than budget
    def find_first_laptop_more_expensive(self, target_price):
        row_start = 0
        row_end = len(self.rows_by_price) - 1
        while row_start < row_end:
            row_middle = (row_end + row_start) // 2
            price = self.rows_by_price[row_middle][-1]
            if price > target_price:
                row_end = row_middle
            else:
                row_start = row_middle + 1
        if self.rows_by_price[row_start][-1] <= target_price:
            return -1
        return row_start

In [10]:
# Test the function
inventory = Inventory('laptops.csv')
print(inventory.find_first_laptop_more_expensive(1000))
print(inventory.find_first_laptop_more_expensive(10000))

683
-1


## Conclusion
This exercise showed how preprocessing data can vastly improve the speed of lookup functions, and how to use a binary search algorithm for a function (in this case, finding how many laptops fall under a given budget).